In [15]:
##Cell-1
import yfinance as yf

In [ ]:
##Cell-2
import pandas as pd
import numpy as np

In [ ]:
##Cell-3
import openpyxl

In [ ]:
##CEll-4
from datetime import datetime

In [ ]:
##Cell-5
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows

In [21]:
##Cell-6
class DCFValuationEngine:
    def __init__(self, ticker='NVDA'):
        self.ticker = ticker
        self.stock = yf.Ticker(ticker)
        self.historical_data = {}
        self.wacc_inputs = {}
        self.forecast_assumptions = {}
        self.forecast_data = {}
        self.dcf_results = {}

In [ ]:
##Cell-7
    def fetch_financial_data(self):
        """Pull financial statements from yfinance"""
        print(f"Fetching financial data for {self.ticker}...")
        
        self.income_stmt = self.stock.financials.T
        self.balance_sheet = self.stock.balance_sheet.T
        self.cashflow_stmt = self.stock.cashflow.T
        
        self.current_price = self.stock.info.get('currentPrice', 0)
        self.shares_outstanding = self.stock.info.get('sharesOutstanding', 0) / 1e6  # in millions
        self.market_cap = self.current_price * self.shares_outstanding * 1e6
        
        print("Financial data fetched successfully!")

In [ ]:
##Cell-8
def clean_and_structure_data(self):
        """Extract and structure key financial metrics"""
        print("Cleaning and structuring data...")
        try:
            revenue = self.income_stmt['Total Revenue'].iloc[-5:]
            ebit = self.income_stmt.get('EBIT', self.income_stmt.get('Operating Income', pd.Series())).iloc[-5:]
            
            da = self.cashflow_stmt.get('Depreciation And Amortization', 
                                        self.cashflow_stmt.get('Depreciation', pd.Series())).iloc[-5:]
            
            capex = -self.cashflow_stmt.get('Capital Expenditure', 
                                            self.cashflow_stmt.get('Capital Expenditures', pd.Series())).iloc[-5:]
            
            current_assets = self.balance_sheet.get('Current Assets', pd.Series()).iloc[-5:]
            current_liabilities = self.balance_sheet.get('Current Liabilities', pd.Series()).iloc[-5:]
            nwc = current_assets - current_liabilities
            delta_nwc = nwc.diff()
            
            tax_expense = self.income_stmt.get('Tax Provision', pd.Series()).iloc[-5:]
            pretax_income = self.income_stmt.get('Pretax Income', pd.Series()).iloc[-5:]
            tax_rate = (tax_expense / pretax_income).mean()
            
            self.historical_data = pd.DataFrame({
                'Revenue': revenue / 1e6,  # in millions
                'EBIT': ebit / 1e6,
                'Tax Rate': tax_rate,
                'NOPAT': (ebit * (1 - tax_rate)) / 1e6,
                'D&A': da / 1e6,
                'CapEx': capex / 1e6,
                'Δ NWC': delta_nwc / 1e6,
            })
            
            self.historical_data['EBITDA'] = self.historical_data['EBIT'] + self.historical_data['D&A']
            self.historical_data['Free Cash Flow'] = (
                self.historical_data['NOPAT'] + 
                self.historical_data['D&A'] - 
                self.historical_data['CapEx'] - 
                self.historical_data['Δ NWC']
            )
            
            self.historical_data['Revenue Growth'] = self.historical_data['Revenue'].pct_change()
            self.historical_data['EBIT Margin'] = self.historical_data['EBIT'] / self.historical_data['Revenue']
            self.historical_data['D&A Ratio'] = self.historical_data['D&A'] / self.historical_data['Revenue']
            self.historical_data['CapEx Ratio'] = self.historical_data['CapEx'] / self.historical_data['Revenue']
            self.historical_data['Working Capital Ratio'] = self.historical_data['Δ NWC'] / self.historical_data['Revenue']
            
            # Base year revenue (most recent year)
            self.base_year_revenue = self.historical_data['Revenue'].iloc[-1]
            
            print("Data cleaned and structured successfully!")
            
        except Exception as e:
            print(f"Error in data cleaning: {e}")
            raise


In [ ]:
    
##Cell-9
def calculate_wacc(self):
        """Calculate WACC using market-based approach"""
        print("Calculating WACC...")

# Get beta from yfinance
        beta = self.stock.info.get('beta', 2.0)
        
        # WACC inputs (matching your model)
        risk_free_rate = 0.04  # 4.0%
        market_risk_premium = 0.05  # 5.0%
        cost_of_debt_pretax = 0.03  # 3.0%
        tax_rate = 0.15  # 15.0%
        
        # Get debt information
        try:
            total_debt = self.balance_sheet['Total Debt'].iloc[-1] / 1e6
        except:
            try:
                total_debt = self.balance_sheet['Long Term Debt'].iloc[-1] / 1e6
            except:
                total_debt = 0
        
        try:
            cash = self.balance_sheet['Cash And Cash Equivalents'].iloc[-1] / 1e6
        except:
            cash = 0
        
        net_debt = total_debt - cash
        
        # Market value of equity
        market_value_equity = self.market_cap / 1e6  # in millions
        
        # Total capital
        total_capital = market_value_equity + net_debt
        
        # Weights
        equity_weight = market_value_equity / total_capital
        debt_weight = net_debt / total_capital
        
        # Cost of equity (CAPM)
        cost_of_equity = risk_free_rate + beta * market_risk_premium
        
        # After-tax cost of debt
        cost_of_debt_aftertax = cost_of_debt_pretax * (1 - tax_rate)
        
        # WACC
        wacc = equity_weight * cost_of_equity + debt_weight * cost_of_debt_aftertax
        
        self.wacc_inputs = {
            'Risk-free rate': risk_free_rate,
            'Beta': beta,
            'Market risk premium': market_risk_premium,
            'Cost of debt (pre-tax)': cost_of_debt_pretax,
            'Tax rate': tax_rate,
            'Equity weight': equity_weight,
            'Debt weight': debt_weight,
            'Cost of equity': cost_of_equity,
            'Cost of debt (after tax)': cost_of_debt_aftertax,
            'WACC': wacc,
        }
        
        self.net_cash = cash - total_debt  # Net cash for valuation
        
        print(f"WACC calculated: {wacc:.2%}")

In [ ]:
##Cell-10
def set_forecast_assumptions(self):
        """Set detailed forecast assumptions matching your model"""
        print("Setting forecast assumptions...")
        
        # Calculate historical averages for ratios
        avg_ebit_margin = self.historical_data['EBIT Margin'].iloc[-3:].mean()
        avg_da_ratio = self.historical_data['D&A Ratio'].iloc[-3:].mean()
        avg_capex_ratio = self.historical_data['CapEx Ratio'].iloc[-3:].mean()
        avg_wc_ratio = self.historical_data['Working Capital Ratio'].iloc[-3:].mean()
        
        # Set forecast assumptions (matching your model structure)
        self.forecast_assumptions = {
            'Base year revenue (FY2025, m)': self.base_year_revenue,
            
            # Year-by-year revenue growth
            'Revenue growth 2026': 0.40,  # 40.0%
            'Revenue growth 2027': 0.308,  # 30.8%
            'Revenue growth 2028': 0.215,  # 21.5%
            'Revenue growth 2029': 0.123,  # 12.3%
            'Revenue growth 2030': 0.09,   # 9.0%
            
            # Operating metrics
            'EBIT margin': avg_ebit_margin,
            'D&A ratio': avg_da_ratio,
            'CapEx ratio': avg_capex_ratio,
            'Working capital ratio': avg_wc_ratio,
            
            # Terminal assumptions
            'Terminal growth rate': 0.03,  # 3.0%
            'Exit EBITDA multiple': 35,
        }
        
        print("Forecast assumptions set successfully!")

In [ ]:
##Cell-11
def generate_forecast(self):
        """Generate 5-year forecast (2026-2030)"""
        print("Generating 5-year forecast...")
        
        years = [2026, 2027, 2028, 2029, 2030]
        revenue_prev = self.forecast_assumptions['Base year revenue (FY2025, m)']
        
        forecast_list = []
        
        for year in years:
            # Revenue forecast
            growth_key = f'Revenue growth {year}'
            growth_rate = self.forecast_assumptions[growth_key]
            revenue = revenue_prev * (1 + growth_rate)
            
            # Income statement items
            ebit = revenue * self.forecast_assumptions['EBIT margin']
            da = revenue * self.forecast_assumptions['D&A ratio']
            ebitda = ebit + da
            
            # Cash flow items
            capex = revenue * self.forecast_assumptions['CapEx ratio']
            delta_nwc = (revenue - revenue_prev) * self.forecast_assumptions['Working capital ratio']
            
            # Free Cash Flow
            nopat = ebit * (1 - self.wacc_inputs['Tax rate'])
            fcf = nopat + da - capex - delta_nwc
            
            forecast_list.append({
                'Year': year,
                'Revenue': revenue,
                'EBIT': ebit,
                'EBITDA': ebitda,
                'D&A': da,
                'CapEx': capex,
                'ΔNWC': delta_nwc,
                'FCF': fcf,
            })
            
            revenue_prev = revenue
        self.forecast_data = pd.DataFrame(forecast_list)
        print("Forecast generated successfully!")
        

In [ ]:
##Cell-12
def calculate_dcf_valuation(self):
        """Calculate DCF valuation with both Gordon Growth and Exit Multiple"""
        print("Calculating DCF valuation...")
        
        wacc = self.wacc_inputs['WACC']
        terminal_growth = self.forecast_assumptions['Terminal growth rate']
        exit_multiple = self.forecast_assumptions['Exit EBITDA multiple']
        
        # Discount factors for years 2026-2030
        years_from_now = [1, 2, 3, 4, 5]
        discount_factors = [(1 + wacc) ** year for year in years_from_now]
        
        # Present value of forecasted FCF
        fcf_values = self.forecast_data['FCF'].values
        pv_fcf = [fcf / df for fcf, df in zip(fcf_values, discount_factors)]
        total_pv_fcf = sum(pv_fcf)
        
        # Method 1: Gordon Growth Terminal Value
        terminal_fcf = fcf_values[-1] * (1 + terminal_growth)
        terminal_value_gordon = terminal_fcf / (wacc - terminal_growth)
        pv_terminal_gordon = terminal_value_gordon / discount_factors[-1]
        enterprise_value_gordon = total_pv_fcf + pv_terminal_gordon
        equity_value_gordon = enterprise_value_gordon + self.net_cash
        value_per_share_gordon = equity_value_gordon / self.shares_outstanding
        
        # Method 2: Exit EBITDA Multiple Terminal Value
        terminal_ebitda = self.forecast_data['EBITDA'].iloc[-1]
        terminal_value_exit = terminal_ebitda * exit_multiple
        pv_terminal_exit = terminal_value_exit / discount_factors[-1]
        enterprise_value_exit = total_pv_fcf + pv_terminal_exit
        equity_value_exit = enterprise_value_exit + self.net_cash
        value_per_share_exit = equity_value_exit / self.shares_outstanding
        
        # Store detailed results
        self.dcf_results = {
            'Year': [2026, 2027, 2028, 2029, 2030],
            'FCF (m)': fcf_values,
            'Disc. factors': discount_factors,
            'PV of FCF (m)': pv_fcf,
            'Total PV FCF': total_pv_fcf,
            
            # Gordon Growth Method
            'Terminal FCF (m)': terminal_fcf,
            'Terminal value (Gordon, m)': terminal_value_gordon,
            'PV Terminal (Gordon, m)': pv_terminal_gordon,
            'Enterprise value (Gordon, m)': enterprise_value_gordon,
            'Equity Value (Gordon)': equity_value_gordon,
            'Value per share (Gordon)': value_per_share_gordon,
            
            # Exit Multiple Method
            'Terminal value (Exit, m)': terminal_value_exit,
            'PV Terminal (Exit, m)': pv_terminal_exit,
            'Enterprise value (Exit, m)': enterprise_value_exit,
            'Equity Value (Exit)': equity_value_exit,
            'Value per share (Exit)': value_per_share_exit,
            
                     # Common items
            'Net cash (m)': self.net_cash,
            'Shares outstanding (m)': self.shares_outstanding,
            'Current Price': self.current_price,
            'Upside (Gordon)': (value_per_share_gordon / self.current_price - 1) * 100,
            'Upside (Exit)': (value_per_share_exit / self.current_price - 1) * 100,
        }
        
        print("DCF valuation calculated successfully!")

In [22]:
##Cell-13
def create_excel_output(self, filename='NVDA_DCF_Model.xlsx'):
        """Create professional Excel output matching your model structure"""
        print(f"Creating Excel output: {filename}...")
        
        wb = openpyxl.Workbook()
        wb.remove(wb.active)
        
        # Styling
        header_fill = PatternFill(start_color='366092', end_color='366092', fill_type='solid')
        header_font = Font(bold=True, color='FFFFFF', size=11)
        title_font = Font(bold=True, size=14, color='000000')
        highlight_fill = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')
        green_fill = PatternFill(start_color='00B050', end_color='00B050', fill_type='solid')
        
        # ===== SHEET 1: WACC & Assumptions =====
        ws1 = wb.create_sheet('Model Assumptions')
        ws1['A1'] = 'NVDA Discounted Cash Flow Model'
        ws1['A1'].font = title_font
        
        # WACC Section
        row = 3
        ws1[f'A{row}'] = 'WACC Assumptions'
        ws1[f'A{row}'].font = Font(bold=True, size=12)
        row += 1
        
        wacc_data = [
            ('Risk-free rate', self.wacc_inputs['Risk-free rate'], '0.0%'),
            ('Beta', self.wacc_inputs['Beta'], '0.0'),
            ('Market risk premium', self.wacc_inputs['Market risk premium'], '0.0%'),
            ('Cost of debt (pre-tax)', self.wacc_inputs['Cost of debt (pre-tax)'], '0.0%'),
            ('Tax rate', self.wacc_inputs['Tax rate'], '0.0%'),
            ('Equity weight', self.wacc_inputs['Equity weight'], '0.0%'),
            ('Debt weight', self.wacc_inputs['Debt weight'], '0.0%'),
            ('Cost of equity', self.wacc_inputs['Cost of equity'], '0.0%'),
            ('Cost of debt (after tax)', self.wacc_inputs['Cost of debt (after tax)'], '0.0%'),
            ('WACC', self.wacc_inputs['WACC'], '0.0%'),
        ]
        
        for label, value, fmt in wacc_data:
            ws1[f'A{row}'] = label
            ws1[f'B{row}'] = value
            if '%' in fmt:
                ws1[f'B{row}'].number_format = '0.0%'
            else:
                ws1[f'B{row}'].number_format = '0.0'
            row += 1
        
        # Forecast Assumptions
        row += 1
        ws1[f'A{row}'] = 'Forecast Assumptions'
        ws1[f'A{row}'].font = Font(bold=True, size=12)
        row += 1
        
        ws1[f'A{row}'] = 'Base year revenue (FY2025, m)'
        ws1[f'B{row}'] = self.forecast_assumptions['Base year revenue (FY2025, m)']
        ws1[f'B{row}'].number_format = '#,##0'
        row += 2
        
        # Revenue growth by year
        ws1[f'A{row}'] = 'Year'
        for i, year in enumerate([2026, 2027, 2028, 2029, 2030], 2):
            ws1.cell(row=row, column=i, value=year)
        row += 1
        
        metrics = [
            ('Revenue growth', [self.forecast_assumptions[f'Revenue growth {y}'] for y in [2026, 2027, 2028, 2029, 2030]], '0.0%'),
            ('EBIT margin', [self.forecast_assumptions['EBIT margin']] * 5, '0.0%'),
            ('D&A ratio', [self.forecast_assumptions['D&A ratio']] * 5, '0.0%'),
            ('CapEx ratio', [self.forecast_assumptions['CapEx ratio']] * 5, '0.0%'),
            ('Working capital ratio', [self.forecast_assumptions['Working capital ratio']] * 5, '0.0%'),
        ]
        
        for label, values, fmt in metrics:
            ws1[f'A{row}'] = label
            for i, val in enumerate(values, 2):
                ws1.cell(row=row, column=i, value=val)
                if '%' in fmt:
                    ws1.cell(row=row, column=i).number_format = '0.0%'
            row += 1
        
        # Terminal assumptions
        row += 1
        ws1[f'A{row}'] = 'Terminal assumptions'
        ws1[f'A{row}'].font = Font(bold=True, size=11)
        row += 1
        ws1[f'A{row}'] = 'Terminal growth rate'
        ws1[f'B{row}'] = self.forecast_assumptions['Terminal growth rate']
        ws1[f'B{row}'].number_format = '0.0%'
        row += 1
        ws1[f'A{row}'] = 'Exit EBITDA multiple'
        ws1[f'B{row}'] = self.forecast_assumptions['Exit EBITDA multiple']
        row += 2
        
        ws1[f'A{row}'] = 'Net cash (m)'
        ws1[f'B{row}'] = self.dcf_results['Net cash (m)']
        ws1[f'B{row}'].number_format = '#,##0'
        row += 1
        ws1[f'A{row}'] = 'Shares outstanding (m)'
        ws1[f'B{row}'] = self.dcf_results['Shares outstanding (m)']
        ws1[f'B{row}'].number_format = '#,##0'
        
        ws1.column_dimensions['A'].width = 30
        ws1.column_dimensions['B'].width = 15
        
        # ===== SHEET 2: DCF Valuation =====
        ws2 = wb.create_sheet('DCF Valuation')
        ws2['A1'] = 'DCF Valuation'
        ws2['A1'].font = title_font
        
        row = 3
        ws2[f'A{row}'] = 'Year'
        for i, year in enumerate(self.dcf_results['Year'], 2):
            ws2.cell(row=row, column=i, value=year)
        ws2.cell(row=row, column=1).font = header_font
        row += 1
        
        # FCF and PV calculations
        fcf_data = [
            ('FCF (m)', self.dcf_results['FCF (m)'], '#,##0'),
            ('Disc. factors', self.dcf_results['Disc. factors'], '0.00000000'),
            ('PV of FCF (m)', self.dcf_results['PV of FCF (m)'], '#,##0'),
        ]
        
        for label, values, fmt in fcf_data:
            ws2[f'A{row}'] = label
            for i, val in enumerate(values, 2):
                ws2.cell(row=row, column=i, value=val)
                ws2.cell(row=row, column=i).number_format = fmt
            row += 1
        
        # Gordon Growth section
        row += 1
        ws2[f'A{row}'] = 'Equity Value (Gordon)'
        ws2[f'B{row}'] = self.dcf_results['Equity Value (Gordon)']
        ws2[f'B{row}'].number_format = '$#,##0'
        row += 1
        
        ws2[f'A{row}'] = 'Terminal FCF (m)'
        ws2[f'B{row}'] = self.dcf_results['Terminal FCF (m)']
        ws2[f'B{row}'].number_format = '#,##0'
        row += 1
        
        ws2[f'A{row}'] = 'Terminal value (Gordon, m)'
        ws2[f'B{row}'] = self.dcf_results['Terminal value (Gordon, m)']
        ws2[f'B{row}'].number_format = '#,##0'
        row += 1
        
        ws2[f'A{row}'] = 'PV Terminal (Gordon, m)'
        ws2[f'B{row}'] = self.dcf_results['PV Terminal (Gordon, m)']
        ws2[f'B{row}'].number_format = '#,##0'
        row += 1
        
        ws2[f'A{row}'] = 'Enterprise value (Gordon, m)'
        ws2[f'B{row}'] = self.dcf_results['Enterprise value (Gordon, m)']
        ws2[f'B{row}'].number_format = '#,##0'
        row += 2
        
        # Exit Multiple section
        ws2[f'A{row}'] = 'Equity Value (Exit)'
        ws2[f'B{row}'] = self.dcf_results['Equity Value (Exit)']
        ws2[f'B{row}'].number_format = '$#,##0'
        row += 1
        
        ws2[f'A{row}'] = 'Terminal value (Exit, m)'
        ws2[f'B{row}'] = self.dcf_results['Terminal value (Exit, m)']
        ws2[f'B{row}'].number_format = '#,##0'
        row += 1
        
        ws2[f'A{row}'] = 'PV Terminal (Exit, m)'
        ws2[f'B{row}'] = self.dcf_results['PV Terminal (Exit, m)']
        ws2[f'B{row}'].number_format = '#,##0'
        row += 1
        
        ws2[f'A{row}'] = 'Enterprise value (Exit, m)'
        ws2[f'B{row}'] = self.dcf_results['Enterprise value (Exit, m)']
        ws2[f'B{row}'].number_format = '#,##0'
        row += 2
        
        # Common items
        ws2[f'A{row}'] = 'Net cash (m)'
        ws2[f'B{row}'] = self.dcf_results['Net cash (m)']
        ws2[f'B{row}'].number_format = '#,##0'
        row += 1
        
        ws2[f'A{row}'] = 'Equity value (Gordon, m)'
        ws2[f'B{row}'] = self.dcf_results['Equity Value (Gordon)']
        ws2[f'B{row}'].number_format = '#,##0'
        row += 1
        
        ws2[f'A{row}'] = 'Equity value (Exit, m)'
        ws2[f'B{row}'] = self.dcf_results['Equity Value (Exit)']
        ws2[f'B{row}'].number_format = '#,##0'
        row += 1
        
        ws2[f'A{row}'] = 'Shares outstanding (m)'
        ws2[f'B{row}'] = self.dcf_results['Shares outstanding (m)']
        ws2[f'B{row}'].number_format = '#,##0'
        row += 2
        
        # Valuation per share
        ws2[f'A{row}'] = 'Value per share (Gordon)'
        ws2[f'B{row}'] = self.dcf_results['Value per share (Gordon)']
        ws2[f'B{row}'].number_format = '$#,##0'
        row += 1
        
        ws2[f'A{row}'] = 'Value per share (Exit)'
        ws2[f'B{row}'] = self.dcf_results['Value per share (Exit)']
        ws2[f'B{row}'].number_format = '$#,##0'
        ws2[f'B{row}'].fill = highlight_fill
        ws2[f'B{row}'].font = Font(bold=True)
        
        ws2.column_dimensions['A'].width = 30
        for col in range(2, 8):
            ws2.column_dimensions[openpyxl.utils.get_column_letter(col)].width = 15
        
        # ===== SHEET 3: Forecast FCF =====
        ws3 = wb.create_sheet('Forecast Free Cash Flow')
        ws3['A1'] = 'Forecast Free Cash Flow'
        ws3['A1'].font = title_font
        
        row = 3
        ws3[f'A{row}'] = 'Year'
        for i, year in enumerate(self.forecast_data['Year'], 2):
            ws3.cell(row=row, column=i, value=year)
            ws3.cell(row=row, column=i).font = header_font
            ws3.cell(row=row, column=i).fill = header_fill
        ws3[f'A{row}'].font = header_font
        ws3[f'A{row}'].fill = header_fill
        row += 1
        
        # Forecast data
        fcf_forecast = [
            ('Revenue', self.forecast_data['Revenue'].values, '#,##0'),
            ('EBIT', self.forecast_data['EBIT'].values, '#,##0'),
            ('EBITDA', self.forecast_data['EBITDA'].values, '#,##0'),
            ('D&A', self.forecast_data['D&A'].values, '#,##0'),
            ('CapEx', self.forecast_data['CapEx'].values, '#,##0'),
            ('ΔNWC', self.forecast_data['ΔNWC'].values, '#,##0'),
            ('FCF', self.forecast_data['FCF'].values, '#,##0'),
        ]
        
        for label, values, fmt in fcf_forecast:
            ws3[f'A{row}'] = label
            ws3[f'A{row}'].font = Font(bold=True)
            for i, val in enumerate(values, 2):
                ws3.cell(row=row, column=i, value=val)
                ws3.cell(row=row, column=i).number_format = fmt
            row += 1
        
        ws3.column_dimensions['A'].width = 20
        for col in range(2, 8):
            ws3.column_dimensions[openpyxl.utils.get_column_letter(col)].width = 18
        
        # ===== SHEET 4: Historical Data =====
        ws4 = wb.create_sheet('Historical Data')
        ws4['A1'] = 'Historical Financial Data'
        ws4['A1'].font = title_font
        
        historical_display = self.historical_data[['Revenue', 'EBIT', 'EBITDA', 'D&A', 'CapEx', 'Δ NWC', 'Free Cash Flow']].copy()
        historical_display.index = [d.strftime('%Y') for d in historical_display.index]
        
        for r_idx, r in enumerate(dataframe_to_rows(historical_display, index=True, header=True), 3):
            for c_idx, value in enumerate(r, 1):
                cell = ws4.cell(row=r_idx, column=c_idx, value=value)
                if r_idx == 3:
                    cell.fill = header_fill
                    cell.font = header_font
                elif c_idx > 1 and r_idx > 3:
                    cell.number_format = '#,##0'
        
        for col in range(1, 9):
            ws4.column_dimensions[openpyxl.utils.get_column_letter(col)].width = 18
        
        wb.save(filename)
        print(f"Excel file created successfully: {filename}")
        



In [27]:

##Cell-14
def run_full_analysis(self, output_file='NVDA_DCF_Model.xlsx'):
        """Execute complete DCF analysis pipeline"""
        print(f"\n{'='*70}")
        print(f"NVIDIA (NVDA) DCF VALUATION MODEL")
        print(f"{'='*70}\n")
        
        self.fetch_financial_data()
        self.clean_and_structure_data()
        self.calculate_wacc()
        self.set_forecast_assumptions()
        self.generate_forecast()
        self.calculate_dcf_valuation()
        self.create_excel_output(output_file)
        
        print(f"\n{'='*60}")
        print("VALUATION RESULTS SUMMARY")
        print(f"{'='*60}")
        print(f"Current Price: ${self.dcf_results['Current Price']:.2f}")
        print(f"\nGordon Growth Method:")
        print(f"  Fair Value: ${self.dcf_results['Value per share (Gordon)']:.2f}")
        print(f"  Upside: {self.dcf_results['Upside (Gordon)']:.1f}%")
        print(f"\nExit Multiple Method:")
        print(f"  Fair Value: ${self.dcf_results['Value per share (Exit)']:.2f}")
        print(f"  Upside: {self.dcf_results['Upside (Exit))']:.1f}%")
        print(f"{'='*60}\n")
        
        return self.dcf_results


In [28]:
##Cell-15
# Attach top-level functions to the class as methods
DCFValuationEngine.clean_and_structure_data = clean_and_structure_data
DCFValuationEngine.calculate_wacc = calculate_wacc
DCFValuationEngine.set_forecast_assumptions = set_forecast_assumptions
DCFValuationEngine.generate_forecast = generate_forecast
DCFValuationEngine.calculate_dcf_valuation = calculate_dcf_valuation
DCFValuationEngine.create_excel_output = create_excel_output
DCFValuationEngine.run_full_analysis = run_full_analysis


In [29]:

# Execute the DCF analysis
if __name__ == "__main__":
    # Create DCF engine instance
    dcf = DCFValuationEngine()
    
    # Run complete analysis
    results = dcf.run_full_analysis('NVDA_DCF_Valuation.xlsx')
    
    print("Analysis complete! Check the Excel file for detailed results.")


NVIDIA (NVDA) DCF VALUATION MODEL



AttributeError: 'DCFValuationEngine' object has no attribute 'fetch_financial_data'